# Azure AI Agents with LangChain Integration

This notebook demonstrates how to use Azure AI Agents with LangChain through the `langchain-azure-ai` integration, powered by the Azure AI Projects SDK.

## Prerequisites

1. An Azure AI Foundry project
2. A deployed model (e.g., gpt-4o-mini)
3. Appropriate authentication credentials
4. Environment variables set:
   - `PROJECT_CONNECTION_ENDPOINT`

Install langchain-azure-ai
`pip install langchain-azure-ai` then you can run the following cells

In [5]:
# Try to import Azure and LangChain modules
from azure.identity import DefaultAzureCredential
# Import other required modules
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from langchain_azure_ai.azure_ai_agents import AzureAIAgentsService

In [ ]:
import os

# Set up environment variables 
# note project connnection string is no longer supported
os.environ["PROJECT_ENDPOINT"] = ""
os.environ["MODEL_DEPLOYMENT_NAME"] = ""

## Basic Usage

Let's start with a basic example of creating an Azure AI Agent and using it with LangChain:

In [7]:
# Create an Azure AI Agents service using Azure AI Projects SDK
agent_service = AzureAIAgentsService(
    credential=DefaultAzureCredential(),
    endpoint=os.environ["PROJECT_ENDPOINT"],  # Use the project endpoint
    model="gpt-4.1",  # Use a model that's available in your project
    agent_name="langchain-demo-agent",
    instructions="You are a helpful AI assistant that provides clear and concise answers.",
)

print("Agent service created successfully!")

Agent service created successfully!


In [8]:
# Test basic generation
response = agent_service.invoke("What is the capital of France?")
print(f"Response: {response}")

Response: The capital of France is Paris.


## Multiple Prompts

You can process multiple prompts at once:

In [9]:
# Generate responses for multiple prompts
prompts = ["What is 2 + 2?", "Name one programming language.", "What color is the sky?"]

results = agent_service.generate(prompts)

for i, generation_list in enumerate(results.generations):
    print(f"Prompt {i+1}: {prompts[i]}")
    print(f"Response: {generation_list[0].text}\n")

Prompt 1: What is 2 + 2?
Response: 2 + 2 = 4

Prompt 2: Name one programming language.
Response: Python is an example of a programming language.

Prompt 3: What color is the sky?
Response: The sky is typically blue during the daytime due to the scattering of sunlight by the atmosphere. At sunrise or sunset, it can appear red, orange, pink, or purple. At night, the sky usually looks dark or black, often dotted with stars. The color can also change depending on weather conditions (such as gray when it’s cloudy).



## Integration with LangChain Chains

The Azure AI Agents service works seamlessly with LangChain chains:

In [10]:
# Create a prompt template
prompt = PromptTemplate(
    input_variables=["topic"], template="Explain {topic} in exactly 3 bullet points."
)

# Create a chain
chain = prompt | agent_service | StrOutputParser()

# Use the chain
result = chain.invoke({"topic": "machine learning"})
print(f"Machine Learning Explanation:\n{result}")

Machine Learning Explanation:
- Machine learning is a field of artificial intelligence where computers learn patterns from data to make predictions or decisions without being explicitly programmed.  
- It involves training algorithms on large datasets to recognize trends, classify information, or forecast outcomes.  
- The main types include supervised learning, unsupervised learning, and reinforcement learning, each suited to different kinds of problems.


In [11]:
# Try with different topics
topics = ["artificial intelligence", "cloud computing", "data science"]

for topic in topics:
    result = chain.invoke({"topic": topic})
    print(f"\n{topic.title()}:")
    print(result)
    print("-" * 50)


Artificial Intelligence:
- Artificial intelligence (AI) is the simulation of human intelligence by machines, especially computers.  
- It enables systems to learn from data, recognize patterns, and make decisions with minimal human intervention.  
- Common applications include virtual assistants, image recognition, language translation, and self-driving cars.
--------------------------------------------------

Cloud Computing:
- Cloud computing delivers computing services (like servers, storage, databases, networking, software) over the internet, often on a pay-as-you-go basis.  
- It allows users to access and manage resources remotely without the need to own or maintain physical hardware.  
- Cloud computing offers scalability, flexibility, and reliability for individuals and businesses of all sizes.
--------------------------------------------------

Data Science:
- Data science combines statistics, programming, and domain knowledge to extract insights from structured and unstructu

## Advanced Configuration

You can customize the agent with various parameters. The Azure AI Agents service supports the following configuration options:

**Supported Parameters:**
- `temperature`: Controls randomness in the output (0.0 to 1.0)
- `top_p`: Controls diversity via nucleus sampling (0.0 to 1.0)  
- `response_format`: Specifies the format that the model must output
- `tools`: List of tools to provide to the agent
- `tool_resources`: Tool resources for the agent
- `metadata`: Custom metadata for the agent

**Note:** Some parameters like `max_completion_tokens`, `max_prompt_tokens`, etc. are not supported by the current Azure AI Agents SDK.

In [12]:
# Create an agent with custom configuration
custom_agent = AzureAIAgentsService(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Use model_name instead of model
    agent_name="custom-configured-agent",
    instructions="You are a creative writing assistant. Write in a poetic style.",
    temperature=0.8,  # Higher temperature for more creativity
    agent_description="A creative writing assistant with poetic flair",
)

# Test the custom agent
creative_response = custom_agent.invoke("Describe a sunset over the ocean.")
print(f"Creative Response:\n{creative_response}")

Creative Response:
The sun, a molten coin, slips toward the seam  
where sky and water hush in evening’s dream.  
Molten gold spills over restless waves,  
each crest a brushstroke, each trough a prayer.  
Clouds, flushed with the afterglow of day,  
wear apricot, rose, and lavender  
braided above the infinite blue.  
Salt air thickens with the hush of foam—  
gulls call softer, shadows lengthen.  
Light pours and pools, then slowly withdraws  
into the velvet waiting of night,  
leaving the ocean rimmed in burnished fire,  
the horizon a secret, smoldering line.


## Tool Usage with Azure AI Agents

Azure AI Agents support various tools including code interpreter, file search, and function calling. Here are examples of how to use these tools with the LangChain integration using the Azure AI Projects SDK.

### Code Interpreter Tool

The code interpreter tool allows agents to write and execute Python code, analyze data, create visualizations, and work with files.

In [ ]:
!pip install pandas 

In [14]:
import os
import tempfile
from pathlib import Path

# First, let's create some sample data for the code interpreter to work with
import pandas as pd

# Create sample sales data
data = {
    "month": ["Jan", "Feb", "Mar", "Apr", "May", "Jun"],
    "sales": [12000, 15000, 18000, 14000, 22000, 25000],
    "region": ["North", "South", "East", "West", "North", "South"],
}

df = pd.DataFrame(data)

# Create a CSV file in the current working directory
csv_path = Path.cwd() / "sample_sales_data.csv"
df.to_csv(
    csv_path, index=False, encoding="utf-8-sig"
)  # The code interpreter requires utf-8-sig

print(f"Created sample data file → {csv_path}")
print("\nSample data:")
print(df)

Created sample data file → /workspaces/langchain-azure/libs/azure-ai/docs/sample_sales_data.csv

Sample data:
  month  sales region
0   Jan  12000  North
1   Feb  15000  South
2   Mar  18000   East
3   Apr  14000   West
4   May  22000  North
5   Jun  25000  South


In [15]:
# Import required modules for tools using Azure AI Projects SDK
from azure.ai.agents.models import (CodeInterpreterTool, FilePurpose,
                                    FileSearchTool, MessageRole)

# from azure.ai.projects import AIProjectClient

try:
    agents_client = AzureAIAgentsService(
        credential=DefaultAzureCredential(),
        endpoint=os.environ["PROJECT_ENDPOINT"],  # Use the project endpoint
        # model_name="gpt-4.1",  # Use a model that's available in your project
    )._create_client()

    # # Upload the file for the code interpreter
    print("Uploading file for code interpreter...")
    uploaded_file = agents_client.agents.files.upload_and_poll(
        file_path=str(csv_path),
        purpose=FilePurpose.AGENTS,  # Specify the purpose for the file
    )

    print(f"✓ Uploaded file, file ID: {uploaded_file.id}")

    # Create code interpreter tool with the uploaded file
    code_interpreter = CodeInterpreterTool(file_ids=[uploaded_file.id])

    print(code_interpreter.definitions)

    print(code_interpreter.resources)

    agent = agents_client.agents.create_agent(
        model="gpt-4.1",
        name="code-interpreter-agent",
        instructions="""You are a data analyst agent. 
                Analyze the provided data and create visualizations 
                when helpful. Use Python code to explore and understand the data.""",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    thread = agents_client.agents.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    print(f"✓ Created AzureAIAgentsService with code interpreter tool")

    # Create a message
    message = agents_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="create a pie chart with the data showing sales by region and show it to me as a png image.",
    )
    print(f"Created message, message ID: {message.id}")

    run = agents_client.agents.runs.create_and_process(
        thread_id=thread.id, agent_id=agent.id
    )
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # [START get_messages_and_save_files]
    messages = agents_client.agents.messages.list(thread_id=thread.id)
    print(f"Messages: {messages}")

    for msg in messages:
        breakpoint()
        # Save every image file in the message
        for img in msg.image_contents:
            file_id = img.image_file.file_id
            file_name = f"{file_id}_image_file.png"
            agents_client.files.save(
                file_id=file_id,
                file_name=file_name,
                target_dir="/workspaces/langchain-azure/libs/azure-ai/docs/",
            )
            print(f"Saved image file to: {Path.cwd() / file_name}")

        # Print details of every file-path annotation
        for ann in msg.file_path_annotations:
            print("File Paths:")
            print(f"  Type: {ann.type}")
            print(f"  Text: {ann.text}")
            print(f"  File ID: {ann.file_path.file_id}")
            print(f"  Start Index: {ann.start_index}")
            print(f"  End Index: {ann.end_index}")
    # [END get_messages_and_save_files]

    last_msg = agents_client.agents.messages.get_last_message_text_by_role(
        thread_id=thread.id, role=MessageRole.AGENT
    )
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please check your Azure AI Projects configuration and try again.")
    import traceback

    traceback.print_exc()

Uploading file for code interpreter...
✓ Uploaded file, file ID: assistant-LeH2k6xbTZULFMv5YrxYqr
[{'type': 'code_interpreter'}]
{'code_interpreter': {'file_ids': ['assistant-LeH2k6xbTZULFMv5YrxYqr']}}
Created agent, agent ID: asst_1K2LnACleXYvFq9Ositq055t
Created thread, thread ID: thread_TMfMEQ8UXi9sKTfzxX8yTJuA
✓ Created AzureAIAgentsService with code interpreter tool
Created message, message ID: msg_bUMDC4YR4fLKCfPVHnx2UG7h
Run finished with status: completed
Messages: <iterator object azure.core.paging.ItemPaged at 0x74fe352fa5b0>
File Paths:
  Type: file_path
  Text: sandbox:/mnt/data/sales_by_region_pie.png
  File ID: assistant-5kueJPNXEsmcGQikEGhgTX
  Start Index: 162
  End Index: 203
Last Message: The pie chart showing sales distribution by region has been generated. You can download the PNG image using the link below:

[Download sales by region pie chart](sandbox:/mnt/data/sales_by_region_pie.png)


In [16]:
# Create a persistent thread for multi-turn conversation
thread = agents_client.agents.threads.create()
print(f"Created thread: {thread.id}")

# Have a multi-turn conversation
conversation = [
    "Hello! I'm learning about Python.",
    "What are the key features of Python?",
    "Can you give me an example of a Python function?",
]

for user_message in conversation:
    # Create message
    message = agents_client.agents.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )

    # Run the agent
    run = agents_client.agents.runs.create_and_process(
        thread_id=thread.id, agent_id=agent.id
    )

    # Get the latest response
    messages = agents_client.agents.messages.list(thread_id=thread.id)

    # Find the latest assistant message
    # ItemPaged is iterable directly, no need for .data
    for msg in messages:
        if msg.role == "assistant" and msg.text_messages:
            assistant_response = msg.text_messages[0].text.value
            break

    print(f"\nUser: {user_message}")
    print(f"Assistant: {assistant_response}")
    print("-" * 80)

# Clean up the thread
agents_client.agents.threads.delete(thread.id)
print("\nThread deleted.")

Created thread: thread_M4x3PVVgMcMAOtrGmTh0NBco

User: Hello! I'm learning about Python.
Assistant: Hello! That’s great—Python is an excellent language to learn, especially for data analysis.

I see you’ve uploaded a file. Would you like to explore this file together using Python? If you let me know your goals or questions, I can guide you through the analysis step by step.
--------------------------------------------------------------------------------

User: What are the key features of Python?
Assistant: Python is popular and widely used, especially for data analysis, due to several key features:

### 1. **Simple and Readable Syntax**
   - Easy to write and understand, which makes it beginner-friendly.
   - Code is clear and approachable.

### 2. **Interpreted Language**
   - Executes code line by line, making debugging and testing easier.

### 3. **Dynamically Typed**
   - No need to specify variable types; Python figures it out at runtime.

### 4. **Extensive Standard Library**
  

## Async Operations

The service also supports async operations:

In [17]:
import asyncio

from azure.identity import DefaultAzureCredential


async def async_example():
    # Create async agent
    async_agent = AzureAIAgentsService(
        endpoint=os.environ["PROJECT_ENDPOINT"],
        credential=DefaultAzureCredential(),
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Use model_name instead of model
        agent_name="async-demo-agent",
        instructions="You are an async AI assistant.",
    )

    try:
        # Async generation
        response = await async_agent.ainvoke(
            "What are the benefits of asynchronous programming?"
        )
        print(f"Async Response: {response}")

        # Async multiple prompts
        prompts = ["Hello", "How are you?", "Goodbye"]
        results = await async_agent.agenerate(prompts)

        print("\nAsync Multiple Responses:")
        for i, generation_list in enumerate(results.generations):
            print(f"{prompts[i]}: {generation_list[0].text}")

    finally:
        # Clean up
        await async_agent.adelete_agent()
        await async_agent.aclose()  # closes aiohttp session


# Run the async example
await async_example()

Async Response: **Asynchronous programming** is a programming paradigm that allows a unit of work to run separately from the main application thread, notifying the main thread when the work is complete. Here are some of the primary benefits:

---

### **1. Improved Responsiveness**
- User interfaces remain responsive, even during lengthy operations (e.g., file I/O, network requests).
- In GUI applications, the UI doesn't freeze when background tasks execute.

---

### **2. Better Resource Utilization**
- The program can initiate multiple tasks without waiting for each to complete.
- System resources (CPU, I/O, etc.) are used more efficiently, avoiding idle time.

---

### **3. Scalability**
- Asynchronous server applications (e.g., web servers) can handle thousands of simultaneous connections efficiently.
- Each request isn't blocked, so fewer threads are needed, which reduces memory and context-switching overhead.

---

### **4. Faster Execution for Some Workloads**
- Multiple I/O-bou

## Cleanup

Always remember to clean up your agents when you're done:

In [18]:
# Clean up all the agents we created
try:
    agent_service.delete_agent()
    print("Deleted main agent")
except:
    print("Main agent already deleted or not found")

try:
    custom_agent.delete_agent()
    print("Deleted custom agent")
except:
    print("Custom agent already deleted or not found")

print("\nCleanup completed!")

Deleted main agent
Deleted custom agent

Cleanup completed!


### Other things to try

- Explore file search capabilities
- Implement custom function calling
- Build multi-tool agents
- Integrate with more complex LangChain workflows

The Azure AI Agents integration provides a powerful way to combine the capabilities of Azure AI Projects with the flexibility of LangChain, enabling you to build sophisticated AI applications with ease.